In [1]:
from OR_based_single_route_op import *
import h3
import numpy as np
import pandas as pd
from db_conn import *
from config import db_config
import h3
from config import config
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import math

In [2]:
WEIGHTS = config.WEIGHTS
TRUCK_CAPACITY = config.TRUCK_CAPACITY
FORMAT_PENALTY = config.FORMAT_PENALTY
TOTAL_DRIVE_TIME = config.TOTAL_DRIVE_TIME

In [3]:
def execute_pg_query(query):
    """
    Executing a specific Postgres query
    :param query:
    :return: The result set.
    """
    connection_pool = PostgresConnectionPool(
        host=db_config.HOST,
        port=db_config.PORT,
        database=db_config.DATABASE,
        user=db_config.USER,
        password=db_config.PASSWORD
    )

    pg_client = PostgresClient(connection_pool)
    res = pg_client.execute_query(query)

    return res

In [70]:
def extract_top3_routes(df):
    top3_routes = (
        df.loc[df.groupby("route_id")["dist_km"].idxmin()]
        .nsmallest(3, "dist_km")
    )
    return top3_routes


def extract_customer_list_from_route(route_id):
    route_query = f"select c.customer_id, c.h3_res_9, c.lat, c.lng, crm.route_id  from locinsights.customer_route_mapping crm join locinsights.customer c " \
                  f"on crm.customer_id = c.customer_id " \
                  f"where crm.route_id = 'USF5DC' limit 10--'{str(route_id)}'"
    print(route_query)
    result = execute_pg_query(route_query)
    print(f"Result == {result}, \n type of result == {type(result)}")
    final_res = []
    for row in result:
        final_res.append({
            "customer_id": row[0],
            "h3_res": row[1],
            "lat": row[2],
            "lon": row[3]
        })
    print(f"final res == {final_res}")
    return pd.DataFrame(final_res)

In [32]:
def extract_new_customer_details(cust_df):
    new_customer = {"customer_name": cust_df["customer_name"], "customer_id": 2000, "lat": cust_df["latitude"], "lon": cust_df["longitude"],
                    "h3_res": h3.latlng_to_cell(cust_df["latitude"], cust_df["longitude"], 9)}
    return pd.DataFrame(new_customer)


In [71]:
def extract_route_dc(route_id):
    dc_query = f"select distinct location_id, lat, lng from locinsights.planned_visit " \
               f"where location_type not ilike '%customer%' and " \
               f"route_id = 'USF5DC' limit 1--'{str(route_id)}'"
    print(dc_query)
    result = execute_pg_query(dc_query)
    final_res = []
    for row in result:
        final_res.append({
            'customer_id': int(row[0]),
            'h3_res': h3.latlng_to_cell(row[1], row[2], 9),
            'lat': row[1],
            'lon': row[2],
            'route_id': route_id
        })
    print("Final dc routes == ", final_res)
    return pd.DataFrame(final_res)

In [7]:
def normalize_h3(h3_cell, target_res):
    res = h3.get_resolution(h3_cell)
    if res == target_res:
        return h3_cell
    elif res > target_res:
        return h3.cell_to_parent(h3_cell, target_res)
    else:
        children = h3.cell_to_children(h3_cell, target_res)
        return list(children)[0]


In [8]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi/2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2) ** 2
    )
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [9]:
def build_h3_distance_matrix(cust_df):
    customers = cust_df["customer_id"].to_list()
    h3_map = dict(zip(cust_df["customer_id"], cust_df["h3_res"]))
    print(f"h3 map == {h3_map}")
    index_map = {cid: i for i, cid in enumerate(customers)}

    n = len(customers)
    matrix = [[0]*n for _ in range(n)]

    for i, c1 in enumerate(customers):
        for j, c2 in enumerate(customers):
            if i == j:
                continue
            try:
                h1 = h3_map[c1]
                h2 = h3_map[c2]
                matrix[i][j] = int(len(h3.grid_path_cells(h1, h2)))
            except:
                lat1, lon1 = h3.cell_to_latlng(h3_map[c1])
                lat2, lon2 = h3.cell_to_latlng(h3_map[c2])
                matrix[i][j] = int(haversine_distance(lat1, lon1, lat2, lon2)/0.190395)

    return matrix, index_map

In [10]:
def create_single_vehicle_data(customers, distance_matrix, route_customers):
    data = {}
    data["customers"] = route_customers
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 1
    data["depot"] = 0
    data["vehicle_capacity"] = TRUCK_CAPACITY
    data["max_time"] = TOTAL_DRIVE_TIME
    return data


In [11]:
def build_routing_model(data, customers, weights):
    manager = pywrapcp.RoutingIndexManager(
        len(data["customers"])-1,
        data["num_vehicles"],
        0
    )

    routing = pywrapcp.RoutingModel(manager)

    def cost_callback(from_idx, to_idx):
        i = from_idx
        j = to_idx

        cost = (
            weights["dwell_time"] * customers[j]["dwell_time"]
            + weights["distance"] * data["distance_matrix"][i][j]
            + weights["pallets"] * customers[j]["avg_pallets"]
            # + weights["format"] * FORMAT_PENALTY[customers[j]["format"]]
        )
        print(f"i == {i}, j == {j}, cost == {int(cost)}")
        return int(cost)

    transit_cb = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

    return routing, manager

In [12]:
def add_capacity_constraint(routing, manager, data, customers):
    def demand_callback(from_idx):
        return customers[from_idx]["avg_pallets"]

    demand_cb = routing.RegisterUnaryTransitCallback(demand_callback)

    routing.AddDimensionWithVehicleCapacity(
        demand_cb,
        0,
        [data["vehicle_capacity"]],
        True,
        "Capacity"
    )

In [13]:
def add_time_dimension(routing, manager, data, customers):
    def time_callback(from_idx, to_idx):
        i = from_idx
        j = to_idx
        return data["distance_matrix"][i][j] + customers[j]["dwell_time"]

    time_cb = routing.RegisterTransitCallback(time_callback)

    routing.AddDimension(
        time_cb,
        0,
        data["max_time"],
        True,
        "Time"
    )

In [14]:
def optimize_single_route(route, customers, distance_matrix, weights):
    print(f"route == {route}, \n customers == {customers}, \n distance_matrix == {distance_matrix}, \n weights == {weights}")
    data = create_single_vehicle_data(customers, distance_matrix, route)

    routing, manager = build_routing_model(data, customers, weights)
    add_capacity_constraint(routing, manager, data, customers)
    add_time_dimension(routing, manager, data, customers)

    params = pywrapcp.DefaultRoutingSearchParameters()
    params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    params.time_limit.seconds = 5

    solution = routing.SolveWithParameters(params)

    if not solution:
        print("No solution found. Returning the original route. ")
        return route

    idx = routing.Start(0)
    optimized_route = []

    while not routing.IsEnd(idx):
        optimized_route.append(
            data["customers"][manager.IndexToNode(idx)]
        )
        idx = solution.Value(routing.NextVar(idx))

    return optimized_route

In [34]:
customer_route_json = '''[
      {
        "customer_name": "Walmart ",
        "address": "1202 S Kirkwood Rd, Kirkwood, MO 63122, United States",
        "latitude": 38.5631543,
        "longitude": -90.4030029,
        "nearest_customers": [
          {
            "customer_id": 2003562112,
            "customer_name": "WALMART SC #2694",
            "route_id": "USF6AO",
            "dist_km": 0.029869383967916503
          }
        ]
      }
    ]'''

In [72]:
cust_df = pd.read_json(customer_route_json)
# print(f"Cust df == {cust_df}")
new_customer = extract_new_customer_details(cust_df)
nearest_cust_df = pd.DataFrame.from_records(cust_df['nearest_customers'].to_list()[0])
sorted_cust_df = nearest_cust_df.sort_values(by=['dist_km'], ascending=[True])
top3_routes = extract_top3_routes(sorted_cust_df)

routes = {}

C:\Users\81252557\AppData\Local\Temp\ipykernel_10376\3832932500.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  cust_df = pd.read_json(customer_route_json)
C:\Users\81252557\PycharmProjects\pep-pcaf-locinsights-actl-est-visits-py\.venv\Lib\site-packages\h3\api\basic_str\__init__.py:186: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return _out_scalar(_cy.latlng_to_cell(lat, lng, res))


In [73]:
top3_routes

,customer_id,customer_name,route_id,dist_km
0,2003562112,WALMART SC #2694,USF6AO,0.029869


In [74]:
for row in top3_routes.itertuples():
    route_id = row.route_id
    route_cust_df = extract_customer_list_from_route(route_id) # combine all the data to dc in one single query
    route_cust_df = pd.concat([route_cust_df, new_customer], ignore_index=True)
    dc_details = extract_route_dc(route_id)
    route_cust_df = pd.concat([dc_details, route_cust_df], ignore_index=True)
    route_cust_df["dwell_time"] = 33
    route_cust_df["avg_pallets"] = 11
    route_cust_df["format"] = 'S'

select c.customer_id, c.h3_res_9, c.lat, c.lng, crm.route_id  from locinsights.customer_route_mapping crm join locinsights.customer c on crm.customer_id = c.customer_id where crm.route_id = 'USF5DC' limit 10--'USF6AO'
Result == [(2003771684, '892773b51cfffff', 47.0241259, -88.5266279, 'USF5DC'), (2004190739, '892746d1653ffff', 46.7879087, -88.4754596, 'USF5DC'), (2003693940, '892773a091bffff', 46.9234662, -88.6262665, 'USF5DC'), (2003732599, '892746d138fffff', 46.7503575, -88.44687683, 'USF5DC')], 
 type of result == <class 'list'>
final res == [{'customer_id': 2003771684, 'h3_res': '892773b51cfffff', 'lat': 47.0241259, 'lon': -88.5266279}, {'customer_id': 2004190739, 'h3_res': '892746d1653ffff', 'lat': 46.7879087, 'lon': -88.4754596}, {'customer_id': 2003693940, 'h3_res': '892773a091bffff', 'lat': 46.9234662, 'lon': -88.6262665}, {'customer_id': 2003732599, 'h3_res': '892746d138fffff', 'lat': 46.7503575, 'lon': -88.44687683}]
select distinct location_id, lat, lng from locinsights.plan

In [75]:
route_cust_df

,customer_id,h3_res,lat,lon,route_id,customer_name,dwell_time,avg_pallets,format
0,1700001452,892773b0adbffff,47.094700,-88.623805,USF6AO,NaN,33,11,S
1,2003771684,892773b51cfffff,47.024126,-88.526628,NaN,NaN,33,11,S
2,2004190739,892746d1653ffff,46.787909,-88.475460,NaN,NaN,33,11,S
3,2003693940,892773a091bffff,46.923466,-88.626266,NaN,NaN,33,11,S
4,2003732599,892746d138fffff,46.750357,-88.446877,NaN,NaN,33,11,S
5,2000,89264625ea7ffff,38.563154,-90.403003,NaN,Walmart,33,11,S


In [76]:
distance_matrix, index_map = build_h3_distance_matrix(route_cust_df)

h3 map == {1700001452: '892773b0adbffff', 2003771684: '892773b51cfffff', 2004190739: '892746d1653ffff', 2003693940: '892773a091bffff', 2003732599: '892746d138fffff', 2000: '89264625ea7ffff'}


In [77]:
route_customers = route_cust_df["customer_id"].to_list()
customers = route_cust_df.to_dict("records")

In [78]:
optimized = optimize_single_route(route_customers, customers, distance_matrix, WEIGHTS)

route == [1700001452, 2003771684, 2004190739, 2003693940, 2003732599, 2000], 
 customers == [{'customer_id': 1700001452, 'h3_res': '892773b0adbffff', 'lat': 47.094700198, 'lon': -88.623804955, 'route_id': 'USF6AO', 'customer_name': nan, 'dwell_time': 33, 'avg_pallets': 11, 'format': 'S'}, {'customer_id': 2003771684, 'h3_res': '892773b51cfffff', 'lat': 47.0241259, 'lon': -88.5266279, 'route_id': nan, 'customer_name': nan, 'dwell_time': 33, 'avg_pallets': 11, 'format': 'S'}, {'customer_id': 2004190739, 'h3_res': '892746d1653ffff', 'lat': 46.7879087, 'lon': -88.4754596, 'route_id': nan, 'customer_name': nan, 'dwell_time': 33, 'avg_pallets': 11, 'format': 'S'}, {'customer_id': 2003693940, 'h3_res': '892773a091bffff', 'lat': 46.9234662, 'lon': -88.6262665, 'route_id': nan, 'customer_name': nan, 'dwell_time': 33, 'avg_pallets': 11, 'format': 'S'}, {'customer_id': 2003732599, 'h3_res': '892746d138fffff', 'lat': 46.7503575, 'lon': -88.44687683, 'route_id': nan, 'customer_name': nan, 'dwell_tim

In [62]:
import folium

def visualize_routes(before_routes, after_routes, customers, route_list):
    # Center map at depot
    depot = customers[0]
    m = folium.Map(location=[depot["lat"], depot["lon"]], zoom_start=12)

    colors = [
        "red", "blue", "green", "purple", "orange",
        "darkred", "darkblue", "darkgreen"
    ]

    # -----------------------
    # BEFORE Optimization
    # -----------------------
    before_group = folium.FeatureGroup(name="Before Optimization")

    for idx, (vehicle, route) in enumerate(before_routes.items()):
        color = colors[idx % len(colors)]

        coords = [
            (customers[route_list.index(c)]["lat"], customers[route_list.index(c)]["lon"])
            for c in route
        ]

        folium.PolyLine(
            coords,
            color=color,
            weight=4,
            opacity=0.7,
            tooltip=f"{vehicle} (Before)"
        ).add_to(before_group)

        for c in route:
            folium.CircleMarker(
                location=(customers[route_list.index(c)]["lat"], customers[route_list.index(c)]["lon"]),
                radius=6,
                popup=f"{vehicle} - {customers[route_list.index(c)]['customer_name']}",
                color=color,
                fill=True
            ).add_to(before_group)

    before_group.add_to(m)

    # -----------------------
    # AFTER Optimization
    # -----------------------
    after_group = folium.FeatureGroup(name="After Optimization")

    for idx, (vehicle, route) in enumerate(after_routes.items()):
        color = colors[idx % len(colors)]

        coords = [
            (customers[route_list.index(c)]["lat"], customers[route_list.index(c)]["lon"])
            for c in route
        ]

        folium.PolyLine(
            coords,
            color=color,
            weight=4,
            opacity=0.9,
            dash_array="5,5",
            tooltip=f"{vehicle} (After)"
        ).add_to(after_group)

        for i, c in enumerate(route):
            folium.Marker(
                location=(customers[route_list.index(c)]["lat"], customers[route_list.index(c)]["lon"]),
                popup=f"{vehicle} - Stop {i} - {customers[route_list.index(c)]['customer_name']}",
                icon=folium.Icon(color="green", icon="ok-sign")
            ).add_to(after_group)

    after_group.add_to(m)

    folium.LayerControl(collapsed=False).add_to(m)
    return m


In [79]:
customers = route_cust_df.to_dict(orient='records')
optmized_routes = {"vehicle": optimized}
route_list = route_cust_df["customer_id"].to_list()
route_map = visualize_routes(optmized_routes, optmized_routes, customers, route_list)

In [80]:
route_map

In [82]:
optmized_routes

{'vehicle': [1700001452, 2003771684, 2004190739, 2003693940, 2003732599, 2000]}

In [84]:
route_cust_df

,customer_id,h3_res,lat,lon,route_id,customer_name,dwell_time,avg_pallets,format
0,1700001452,892773b0adbffff,47.094700,-88.623805,USF6AO,NaN,33,11,S
1,2003771684,892773b51cfffff,47.024126,-88.526628,NaN,NaN,33,11,S
2,2004190739,892746d1653ffff,46.787909,-88.475460,NaN,NaN,33,11,S
3,2003693940,892773a091bffff,46.923466,-88.626266,NaN,NaN,33,11,S
4,2003732599,892746d138fffff,46.750357,-88.446877,NaN,NaN,33,11,S
5,2000,89264625ea7ffff,38.563154,-90.403003,NaN,Walmart,33,11,S


In [88]:
final_res = []
temp_dict = route_cust_df.to_dict(orient='records')
for item in temp_dict:
    final_res.append({
        'customer_id': item['customer_id'],
        'customer_name': item['customer_name'],
        'latitude': item['lat'],
        'longitude': item['lon'],
        'route_id': item['route_id'],
        'sequence': optmized_routes['vehicle'].index(item['customer_id'])
    })
final_res

[{'customer_id': 1700001452,
  'customer_name': nan,
  'latitude': 47.094700198,
  'longitude': -88.623804955,
  'route_id': 'USF6AO',
  'sequence': 0},
 {'customer_id': 2003771684,
  'customer_name': nan,
  'latitude': 47.0241259,
  'longitude': -88.5266279,
  'route_id': nan,
  'sequence': 1},
 {'customer_id': 2004190739,
  'customer_name': nan,
  'latitude': 46.7879087,
  'longitude': -88.4754596,
  'route_id': nan,
  'sequence': 2},
 {'customer_id': 2003693940,
  'customer_name': nan,
  'latitude': 46.9234662,
  'longitude': -88.6262665,
  'route_id': nan,
  'sequence': 3},
 {'customer_id': 2003732599,
  'customer_name': nan,
  'latitude': 46.7503575,
  'longitude': -88.44687683,
  'route_id': nan,
  'sequence': 4},
 {'customer_id': 2000,
  'customer_name': 'Walmart ',
  'latitude': 38.5631543,
  'longitude': -90.4030029,
  'route_id': nan,
  'sequence': 5}]

In [90]:
import json
fin_json = json.dumps(final_res)
print(f"final json == {fin_json}, \n type == {type(fin_json)}")

final json == [{"customer_id": 1700001452, "customer_name": NaN, "latitude": 47.094700198, "longitude": -88.623804955, "route_id": "USF6AO", "sequence": 0}, {"customer_id": 2003771684, "customer_name": NaN, "latitude": 47.0241259, "longitude": -88.5266279, "route_id": NaN, "sequence": 1}, {"customer_id": 2004190739, "customer_name": NaN, "latitude": 46.7879087, "longitude": -88.4754596, "route_id": NaN, "sequence": 2}, {"customer_id": 2003693940, "customer_name": NaN, "latitude": 46.9234662, "longitude": -88.6262665, "route_id": NaN, "sequence": 3}, {"customer_id": 2003732599, "customer_name": NaN, "latitude": 46.7503575, "longitude": -88.44687683, "route_id": NaN, "sequence": 4}, {"customer_id": 2000, "customer_name": "Walmart ", "latitude": 38.5631543, "longitude": -90.4030029, "route_id": NaN, "sequence": 5}], 
 type == <class 'str'>
